###  CNN MicroGrad demo

In [1]:
from micrograd.engine import Value
from micrograd.nn import Convolution, CNN, MLP, Neuron
from keras.datasets import mnist
import numpy as np

2023-12-02 09:59:51.914231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 09:59:53.218055: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-02 09:59:53.218164: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-02 09:59:56.109027: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [3]:
train_X = np.expand_dims(train_X, axis=1)
test_X = np.expand_dims(test_X, axis=1)
print(train_X.shape)
print(test_X.shape)

(60000, 1, 28, 28)
(10000, 1, 28, 28)


In [57]:
mlp = MLP(20*20, [10])
model = CNN([Convolution(1, 1, kernel_size=9)], mlp, is_softmax=True)
# model = CNN([Convolution(1, 3, kernel_size=5), Convolution(3, 1, kernel_size=5)], mlp, is_softmax=True)
# 28 - 5 + 1 = 24
# 24 - 5 + 1 = 20

In [58]:
print("number of parameters", len(model.parameters()))

number of parameters 4092


In [95]:
# loss function
def loss(X, y, batch_size=None):

    if batch_size:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]
    else:
        Xb, yb = X, y

    probs = model(Xb)
    if len(probs.shape) == 1:
        probs = np.expand_dims(probs, axis=0)
    y_pred = np.argmax(probs, axis=1)

    y_vec = np.eye(10)[yb]
    if len(y_vec.shape) == 1:
        y_vec = np.expand_dims(y_vec, axis=0)

    losses = np.array([(yi - probi)**2 for yi, probi in zip(y_vec, probs)])
    data_loss = np.sum(losses, axis=1) * (1.0 / len(losses))
    all_samples_loss = np.sum(data_loss) * (1.0 / len(data_loss))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = all_samples_loss + reg_loss

    if not isinstance(yb, np.ndarray):
        yb = np.array([yb])

    # accuracy
    accuracy = [yi == y_predi for yi, y_predi in zip(yb, y_pred)]
    return total_loss, sum(accuracy) / len(accuracy)

In [94]:
# optimization
for k in range(100):
    
    # forward
    total_loss, acc = loss(train_X, train_y, batch_size=32)

    # backward
    model.zero_grad()
    total_loss.backward()
    
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    
    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")


step 0 loss 0.5177250863151045, accuracy 0.0%
step 1 loss 0.49209703232383295, accuracy 0.0%
step 2 loss 0.5471520891032092, accuracy 0.0%
step 3 loss 0.5264596884145148, accuracy 0.0%
step 4 loss 0.5821447914273361, accuracy 0.0%
step 5 loss 0.5763286852951627, accuracy 0.03125%
step 6 loss 0.48278164656089206, accuracy 0.03125%
step 7 loss 0.5436353818943371, accuracy 0.03125%
step 8 loss 0.5193593455445548, accuracy 0.03125%
step 9 loss 0.4746080962051649, accuracy 0.03125%
step 10 loss 0.4856993136840738, accuracy 0.0625%
step 11 loss 0.47849744948023887, accuracy 0.0625%
step 12 loss 0.48000336078772105, accuracy 0.09375%
step 13 loss 0.5328158623305367, accuracy 0.09375%
step 14 loss 0.5213781123733333, accuracy 0.09375%
step 15 loss 0.5249376561833168, accuracy 0.125%
step 16 loss 0.5404190669995745, accuracy 0.125%
step 17 loss 0.4979462909578527, accuracy 0.15625%
step 18 loss 0.5075477590956398, accuracy 0.15625%
step 19 loss 0.48031984501557606, accuracy 0.1875%
step 20 loss